In [1]:
import pandas as pd
import numpy as np
import random
import re

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

from joblib import parallel_backend
import nltk

In [3]:
# Baixando os recursos necessários do NLTK
print("🔄 Baixando os recursos necessários do NLTK...")
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
print("✅ Recursos baixados com sucesso!\n")


def preprocess_text(text):
    """
    Pré-processa o texto de entrada realizando várias operações de limpeza.

    Args:
    text (str): O texto de entrada a ser pré-processado.

    Returns:
    str: O texto pré-processado.
    """
    if pd.isna(text):
        return ''
    if isinstance(text, str):
        # Converter para minúsculas
        text = text.lower()

        # Remover URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Remover números e pontuações, exceto '!' e '?'
        text = re.sub(r'[^\w\s!?]', '', text)

        # Remover espaços em branco extras
        text = re.sub(r'\s+', ' ', text).strip()

        # Tokenização
        tokens = word_tokenize(text.lower())

        # Definir palavras de parada a serem mantidas (incluindo palavras relacionadas a emoções)
        stop_words = set(stopwords.words('english')) - {
            'not', 'no', 'nor', 'but', 'however', 'okay', 'although', 'though',
            'happy', 'sad', 'angry', 'frustrated', 'excited', 'disappointed',
            # ... (resto das palavras relacionadas a emoções)
        }

        # Filtrar tokens
        filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
        return ' '.join(filtered_tokens)
    return ''


def get_synonyms(word):
    """
    Obtém sinônimos para uma palavra dada usando o WordNet.

    Args:
    word (str): A palavra de entrada para encontrar sinônimos.

    Returns:
    list: Uma lista de sinônimos para a palavra de entrada.
    """
    return list({lemma.name() for syn in wordnet.synsets(word) for lemma in syn.lemmas()})


def data_augmentation(text, sentiment, num_augmented=3, substitution_prob=0.3):
    """
    Realiza aumento de dados no texto de entrada por substituição de sinônimos e embaralhamento de palavras.

    Args:
    text (str): O texto de entrada para aumentar.
    sentiment (str): O rótulo de sentimento do texto de entrada.
    num_augmented (int): Número de amostras aumentadas a serem geradas.
    substitution_prob (float): Probabilidade de substituir uma palavra por seu sinônimo.

    Returns:
    list: Uma lista de tuplas contendo textos aumentados e seus sentimentos.
    """
    words = word_tokenize(text)
    augmented_texts = []

    # Substituição por sinônimos
    for _ in range(num_augmented):
        new_text = []
        for word in words:
            synonyms = get_synonyms(word)
            if synonyms and random.random() < substitution_prob:
                new_text.append(random.choice(synonyms))
            else:
                new_text.append(word)
        augmented_texts.append((' '.join(new_text), sentiment))

    # Embaralhamento da ordem das palavras
    for _ in range(num_augmented):
        new_words = words.copy()
        random.shuffle(new_words)
        augmented_texts.append((' '.join(new_words), sentiment))

    return augmented_texts


def load_data(filepath):
    """
    Carrega dados de um arquivo CSV.

    Args:
    filepath (str): O caminho para o arquivo CSV.

    Returns:
    pandas.DataFrame ou None: Os dados carregados como um DataFrame, ou None se ocorrer um erro.
    """
    try:
        column_names = ['id', 'entity', 'sentiment', 'text']
        df = pd.read_csv(filepath, header=None, names=column_names)
        print("✅ Dados carregados com sucesso.")
        return df
    except Exception as e:
        print(f"❌ Erro ao carregar o dataset: {e}")
        return None


def print_divider():
    """Imprime uma linha divisória para melhor legibilidade da saída."""
    print("\n" + "="*60)


def main():
    """
    Função principal para executar o pipeline de análise de sentimentos.
    """
    print("🔍 Iniciando o programa...")

    # Carregando os dados de treinamento
    df_train = load_data("/content/drive/MyDrive/Trabalho IA /twitter_training.csv")

    if df_train is not None:
        print_divider()

        # Explorando os dados de treinamento
        print("📊 Nomes das colunas de treinamento:")
        print(df_train.columns)
        print_divider()
        print("📄 Amostra dos dados de treinamento:")
        print(df_train.head())

        if 'sentiment' in df_train.columns:
            print_divider()
            print("📈 Distribuição das classes de sentimento de treinamento:")
            print(df_train['sentiment'].value_counts())
        else:
            print("❌ Erro: A coluna 'sentiment' não foi encontrada no DataFrame de treinamento.")
            return

        # Remover linhas com valores NaN
        df_train = df_train.dropna(subset=['text', 'sentiment'])

        # Aplicar aumento de dados
        augmented_data = []
        for _, row in df_train.iterrows():
            augmented_data.extend(data_augmentation(row['text'], row['sentiment'], num_augmented=3, substitution_prob=0.3))

        # Adicionar dados aumentados ao DataFrame original
        df_augmented = pd.DataFrame(augmented_data, columns=['text', 'sentiment'])
        df_train = pd.concat([df_train, df_augmented], ignore_index=True)

        # Embaralhar o DataFrame
        df_train = df_train.sample(frac=1).reset_index(drop=True)

        # Pré-processamento dos dados de treinamento
        df_train['clean_text'] = df_train['text'].apply(preprocess_text)

        # Remover linhas vazias após o pré-processamento
        df_train = df_train[df_train['clean_text'] != '']

        # Separação dos dados em características e alvo
        X_train = df_train['clean_text']
        y_train = df_train['sentiment']

        # Criando o pipeline do modelo de PLN usando Naive Bayes
        model = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', MultinomialNB())
        ])

        # Definindo o grid de parâmetros para ajuste de hiperparâmetros
        parameters = {
            'tfidf__ngram_range': [(1,2)],
            'tfidf__max_features': [25000],
            'clf__alpha': [0.3],
        }

        # Realizando Grid Search para ajuste de hiperparâmetros
        grid_search = GridSearchCV(model, parameters, cv=10, n_jobs=-1, verbose=1)

        # Usando parallel_backend para multi-threading
        with parallel_backend('threading'):
            grid_search.fit(X_train, y_train)

        # Obtendo o melhor modelo e parâmetros
        best_model = grid_search.best_estimator_
        print_divider()
        print(f"🔧 Melhores Parâmetros: {grid_search.best_params_}")

        # Realizando validação cruzada com o melhor modelo
        cv_scores = cross_val_score(best_model, X_train, y_train, cv=10)
        print_divider()
        print("🔍 Pontuações da Validação Cruzada:")
        print(cv_scores)
        print(f"🔢 Pontuação média da Validação Cruzada: {np.mean(cv_scores):.4f}")

        # Carregando os dados de validação
        df_val = load_data("/content/drive/MyDrive/Trabalho IA /twitter_validation.csv")

        if df_val is not None:
            # Pré-processamento dos dados de validação
            df_val['text'] = df_val['text'].fillna('').astype(str)
            df_val['clean_text'] = df_val['text'].apply(preprocess_text)

            # Separação dos dados de validação
            X_val = df_val['clean_text']
            y_val = df_val['sentiment']

            # Avaliação do modelo com o conjunto de validação
            y_pred_val = best_model.predict(X_val)
            print_divider()
            print("📈 Relatório de Classificação no conjunto de validação:")
            print(metrics.classification_report(y_val, y_pred_val))

            # Matriz de confusão no conjunto de validação
            conf_matrix_val = metrics.confusion_matrix(y_val, y_pred_val)
            print_divider()
            print("📊 Matriz de Confusão no conjunto de validação:")
            print(conf_matrix_val)

        # Realizando testes com frases de exemplo
        exemplo_frases = [
            "I love this product!",
            "This is the worst experience I've ever had.",
            "It's okay, not bad.",
            "Absolutely fantastic!",
            "I hate it so much.",
            "The game is a bit boring.",
            "I'm thrilled about this new update!",
            "The movie was quite mediocre.",
            "This is the best purchase I've ever made.",
            "I'm really disappointed with the service.",
            "The book was an interesting read.",
            "I feel neutral about this feature.",
            "This restaurant has excellent food.",
            "The app crashes frequently, very frustrating.",
            "I'm excited about the new release!",
            "The product arrived late and damaged.",
            "I enjoyed the concert a lot.",
            "The experience was quite underwhelming.",
            "I'm happy with my new phone.",
            "The software update was a huge improvement.",
            "I'm not satisfied with the customer support.",
            "The quality of the product exceeded my expectations.",
            "The weather was terrible during my vacation.",
            "I had an amazing time at the event.",
            "The service was slow but the food was good.",
            "I'm not impressed with the new design.",
            "The movie was entertaining and engaging.",
            "I felt let down by the recent changes.",
            "The trip was okay, nothing special.",
            "I love the new features in the latest version.",
            "The product is okay but could be better.",
            "The restaurant ambiance was lovely.",
            "I'm frustrated with the frequent bugs.",
            "The book was a great read, highly recommended!",
            "The service was excellent and prompt.",
            "The new update made everything worse.",
            "I feel indifferent about the new changes.",
            "The performance was outstanding.",
            "The quality did not meet my expectations.",
            "I am very pleased with the purchase.",
            "The user interface is much improved now.",
            "The concert was an unforgettable experience.",
            "The game is too repetitive and boring.",
            "I am thrilled with the customer service!",
            "The hotel was clean but the location was poor.",
            "I'm dissatisfied with the recent upgrade.",
            "The movie was a complete waste of time.",
            "The new product features are amazing!"
        ]

        predicoes = best_model.predict(exemplo_frases)
        print_divider()
        print("📝 Testes com frases de exemplo:")
        for frase, sentimento in zip(exemplo_frases, predicoes):
            print(f'Frase: "{frase}"\nSentimento Predito: {sentimento}\n')

        # Análise crítica do modelo
        print_divider()
        print("🔍 Análise Crítica do Modelo:")
        print("Pontos fortes:")
        print("- O modelo utiliza técnicas de pré-processamento e aumento de dados para melhorar o desempenho.")
        print("- A implementação de Grid Search permite otimização de hiperparâmetros.")
        print("- O uso de validação cruzada fornece uma avaliação mais robusta do modelo.")
        print("\nPontos fracos:")
        print("- A detecção de sentimentos sutis ou ambíguos pode ainda ser um desafio.")
        print("- O modelo atual pode ter dificuldades com sarcasmo ou contextos mais complexos.")
        print("- A dependência de um conjunto de dados específico pode limitar a generalização.")
        print("\nOportunidades de melhorias:")
        print("- Explorar modelos mais avançados, como redes neurais ou transformers.")
        print("- Implementar técnicas de análise de erros para identificar padrões de falhas do modelo.")
        print("- Considerar a inclusão de análise de contexto ou informações adicionais para melhorar a precisão.")
        print("- Expandir o conjunto de dados de treinamento com mais exemplos variados e complexos.")

        # Informações adicionais
        print_divider()
        print("ℹ️ Informações adicionais:")
        print("Dados e pré-processamento:")
        print("- Os dados foram aumentados usando técnicas de aumento de dados.")
        print("- O pré-processamento incluiu remoção de URLs, números e pontuações.")
        print("- Stopwords foram removidas, exceto palavras importantes para análise de sentimento.")
        print("- Todos os textos foram convertidos para minúsculas para padronização.")

        print("\nModelo e treinamento:")
        print("- Utilizou-se um pipeline com TfidfVectorizer e MultinomialNB.")
        print("- O Grid Search foi aplicado para otimização de hiperparâmetros.")
        print("- A validação cruzada com 20 folds foi usada para avaliar o desempenho do modelo.")

        print("\nAvaliação e teste:")
        print("- Um conjunto de validação separado foi usado para testar o modelo final.")
        print("- Foram realizados testes com frases de exemplo para verificar o desempenho em casos variados.")

        print("\nLimitações e escopo:")
        print("- O modelo atual suporta apenas análises em inglês.")
        print("- O desempenho pode variar dependendo do contexto e complexidade das frases.")
        print("- Sentimentos sutis ou ambíguos podem apresentar desafios para o modelo.")
    else:
        print("❌ Não foi possível carregar os dados. Verifique o caminho do arquivo e tente novamente.")

if __name__ == "__main__":
    main()

🔄 Baixando os recursos necessários do NLTK...
✅ Recursos baixados com sucesso!

🔍 Iniciando o programa...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ Dados carregados com sucesso.

📊 Nomes das colunas de treinamento:
Index(['id', 'entity', 'sentiment', 'text'], dtype='object')

📄 Amostra dos dados de treinamento:
     id       entity sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                text  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  

📈 Distribuição das classes de sentimento de treinamento:
sentiment
Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64
Fitting 10 folds for each of 1 candidates, totalling 10 fits

🔧 Melhores Parâmetros: {'clf__alpha': 0.3, 'tfidf__max_feature